In [1]:
import urllib
import time

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
stockCode = '003530' # 065450 빅텍

dayPriceUrl = 'http://finance.naver.com/item/sise_day.nhn?code=' + stockCode
dayPriceHtml = urlopen(dayPriceUrl)
dayPriceSource = BeautifulSoup(dayPriceHtml.read(), "html.parser")

dayPricePageNavigation = dayPriceSource.find_all("table", align="center")
dayPriceMaxPageSection = dayPricePageNavigation[0].find_all("td", class_="pgRR")
dayPriceMaxPageNum = int(dayPriceMaxPageSection[0].a.get('href')[-3:])

In [40]:
dayPriceMaxPageNum

567

In [41]:
url = 'https://finance.naver.com/item/sise_day.nhn?code=003530&page=2'
html = urlopen(url)
source = BeautifulSoup(html.read(), "html.parser")
srlists = source.find_all("tr")
isCheckNone = None
print(srlists)

[<tr>
<th>날짜</th>
<th>종가</th>
<th>전일비</th>
<th>시가</th>
<th>고가</th>
<th>저가</th>
<th>거래량</th>
</tr>, <tr>
<td colspan="7" height="8"></td>
</tr>, <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2018.12.11</span></td>
<td class="num"><span class="tah p11">2,055</span></td>
<td class="num">
<img alt="하락" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_down.gif" style="margin-right:4px;" width="7"/><span class="tah p11 nv01">
				25
				</span>
</td>
<td class="num"><span class="tah p11">2,080</span></td>
<td class="num"><span class="tah p11">2,105</span></td>
<td class="num"><span class="tah p11">2,055</span></td>
<td class="num"><span class="tah p11">131,051</span></td>
</tr>, <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2018.12.10</span></td>
<td class="num"><span class="tah p11">2,080</span></td>
<td class="num">
<img alt="하락" height="6" sr

In [55]:
print(srlists[3].span)
print(srlists[3].find_all("td", align="center")[0].text)
print(srlists[3].find_all("td", class_="num")[0].text)

<span class="tah p10 gray03">2018.12.10</span>
2018.12.10
2,080


In [56]:
day_list = []
close = []
open_list = []
high = []
low = []
vol = []
for page in range(1, dayPriceMaxPageNum + 1):
    url = 'http://finance.naver.com/item/sise_day.nhn?code=' + stockCode + '&page=' + str(page)
    html = urlopen(url)
    source = BeautifulSoup(html.read(), "html.parser")
    srlists = source.find_all("tr")
    isCheckNone = None

    # day: 날짜
    # closingPrice: 종가
    # variation: 전일대비
    # openingPrice: 시가
    # highestPrice: 고가
    # lowestPrice: 저가
    # volume: 거래량

    for i in range(1, len(srlists) - 1): 
        if (srlists[i].span != isCheckNone):
            day = srlists[i].find_all("td", align="center")[0].text
            closingPrice = srlists[i].find_all("td", class_="num")[0].text
            openingPrice = srlists[i].find_all("td", class_="num")[2].text
            highestPrice = srlists[i].find_all("td", class_="num")[3].text
            lowestPrice = srlists[i].find_all("td", class_="num")[4].text
            volume = srlists[i].find_all("td", class_="num")[5].text

#             print("날짜: " + day, end=" ")
#             print("종가: " + closingPrice, end=" ")
#             print("시가: " + openingPrice, end=" ")
#             print("고가: " + highestPrice, end=" ")
#             print("저가: " + lowestPrice, end=" ")
#             print("거래량: " + volume)
            day_list.append(day)
            close.append(closingPrice)
            open_list.append(openingPrice)
            high.append(highestPrice)
            low.append(lowestPrice)
            vol.append(volume)

In [57]:
len(day_list)

5666

In [14]:
import pandas as pd

In [23]:
raw_data = {
    'date' : day_list,
    'closingPrice' : close,
    'openingPrice' : open_list,
    'highestPrice' : high,
    'lowestPrice' : low,
    'volume' : vol
}

In [24]:
df = pd.DataFrame(raw_data)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 6 columns):
date            186 non-null object
closingPrice    186 non-null object
openingPrice    186 non-null object
highestPrice    186 non-null object
lowestPrice     186 non-null object
volume          186 non-null object
dtypes: object(6)
memory usage: 8.8+ KB


In [28]:
df.head(10)

,date,closingPrice,openingPrice,highestPrice,lowestPrice,volume
0,2018.12.12,"2,105","2,055","2,105","2,045","253,293"
1,2018.11.28,"2,165","2,160","2,185","2,135","125,793"
2,2018.11.14,"2,165","2,140","2,185","2,125","219,157"
3,2018.10.31,"2,040","1,970","2,055","1,955","314,151"
4,2018.10.17,"2,125","2,120","2,150","2,105","371,494"
5,2018.10.01,"2,600","2,595","2,625","2,580","245,149"
6,2018.09.12,"2,490","2,530","2,530","2,480","148,610"
7,2018.08.29,"2,580","2,580","2,600","2,560","312,621"
8,2018.08.14,"2,405","2,400","2,425","2,395","385,139"
9,2018.07.31,"2,535","2,485","2,550","2,470","369,926"


In [34]:
df.sort_values(by=df['date'])

KeyError: 0     2018-12-12
1     2018-11-28
2     2018-11-14
3     2018-10-31
4     2018-10-17
5     2018-10-01
6     2018-09-12
7     2018-08-29
8     2018-08-14
9     2018-07-31
10    2018-07-17
11    2018-07-03
12    2018-06-19
13    2018-06-01
14    2018-05-17
15    2018-05-02
16    2018-04-17
17    2018-04-03
18    2018-03-20
19    2018-03-06
20    2018-02-19
21    2018-02-01
22    2018-01-18
23    2018-01-04
24    2017-12-18
25    2017-12-04
26    2017-11-20
27    2017-11-06
28    2017-10-23
29    2017-09-29
         ...    
156   2012-08-07
157   2012-07-24
158   2012-07-10
159   2012-06-26
160   2012-06-12
161   2012-05-25
162   2012-05-11
163   2012-04-26
164   2012-04-12
165   2012-03-28
166   2012-03-14
167   2012-02-28
168   2012-02-14
169   2012-01-31
170   2012-01-13
171   2011-12-29
172   2011-12-15
173   2011-12-01
174   2011-11-17
175   2011-11-03
176   2011-10-20
177   2011-10-06
178   2011-09-21
179   2011-09-05
180   2011-08-22
181   2011-08-05
182   2011-07-22
183 

In [30]:
idx = pd.to_datetime(df['date'])

In [33]:
df['date'] =idx

In [ ]:
df.

In [36]:
df.sort_values(by=['date'], inplace=True)

In [38]:
df.head(50)

,date,closingPrice,openingPrice,highestPrice,lowestPrice,volume
185,2011-06-10,"5,960","6,110","6,110","5,960","153,890"
184,2011-06-24,"6,150","6,080","6,200","6,030","211,896"
183,2011-07-08,"6,570","6,670","6,680","6,540","157,160"
182,2011-07-22,"6,560","6,510","6,620","6,450","238,030"
181,2011-08-05,"5,820","5,720","5,880","5,700","360,109"
180,2011-08-22,"5,030","5,060","5,250","5,000","306,219"
179,2011-09-05,"5,380","5,510","5,550","5,360","179,403"
178,2011-09-21,"5,240","5,280","5,280","5,170","173,683"
177,2011-10-06,"4,800","4,790","4,890","4,750","234,110"
176,2011-10-20,"4,990","5,140","5,180","4,990","253,871"
